In [1]:
%pwd

'd:\\Comapy\\Projects\\CureNow\\research'

In [9]:
import os


os.chdir("../")

In [10]:
%pwd

'd:\\Comapy\\Projects\\CureNow'

# Load Environment Variables

In [1]:
from dotenv import load_dotenv
import os

load_dotenv()

GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")

In [2]:
print("HI")

HI


# Import Necessary Libraries

In [3]:
import google.generativeai as genai
from langchain.document_loaders import PyPDFLoader,DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_pinecone import PineconeVectorStore
from pinecone import Pinecone,ServerlessSpec

c:\Users\ASUS\anaconda3\envs\curehealth\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Load PDF Data

In [6]:
#Extract the data from the pdf

def load_pdf_file(data):
    loader = DirectoryLoader(data,
                             glob="*.pdf",
                             loader_cls=PyPDFLoader)
    
    documents = loader.load()

    return documents
extracted_data = load_pdf_file(data="Data/")

FileNotFoundError: Directory not found: 'Data/'

# Split Text into Chunks

In [ ]:
# pip list - to see the downloaded version of the library
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size = 1000,chunk_overlap = 100)
    text_chunks = text_splitter.split_documents(extracted_data)
    return text_chunks

text_chunks = text_split(extracted_data)
print(f"Length of text chunks = {len(text_chunks)}")

# Generate Embedding

In [ ]:
def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

embeddings = download_hugging_face_embeddings()
query_result = embeddings.embed_query("Hello World!")
print("Length",len(query_result))

# Initialize Pinecone

In [11]:
PINECONE_API_KEY = os.environ.get('PINECONE_API_KEY')
# OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY")
GEMINI_API_KEY = os.environ.get("GEMINI_API_KEY")

In [ ]:
from pinecone import Pinecone, ServerlessSpec

pc = Pinecone(api_key=PINECONE_API_KEY)
index_name = "curenow"

pc.create_index(
    name=index_name,
    dimension=384, # Replace with your model dimensions
    metric="cosine", # Replace with your model metric
    spec=ServerlessSpec(
        cloud="aws",
        region="us-east-1"
    ) 
)

In [158]:
os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY
# os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY
os.environ["GEMINI_API_KEY"] = GEMINI_API_KEY

In [ ]:
#Initializing the Pinecone
Pinecone(api_key=PINECONE_API_KEY,environment = "us-east-1"
              )

# Store Data in Pinecone

In [ ]:
from langchain_pinecone import PineconeVectorStore

docresearch = PineconeVectorStore.from_documents(
    documents=text_chunks,
    index_name=index_name,
    embedding=embeddings,
)


# Load Exiting Index

In [ ]:
#Load Existing index

from langchain_pinecone import PineconeVectorStore

#embed each chunk and upsert the embeddings into your Pinecone index.

docresearch = PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embeddings
)

In [162]:
retriever = docresearch.as_retriever(search_type = 'similarity',search_kwargs = {"k" : 3})


In [ ]:
retrieved_docs = retriever.invoke("What is Acne?")


In [ ]:
retrieved_docs

In [185]:
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "cure-450112-a6ef848507da.json"


In [186]:
# from langchain_openai import OpenAI

# llm = OpenAI(temperature=0.4,max_tokens=400)
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash-lite-preview-02-05",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    # other params...
)

# https://python.langchain.com/docs/integrations/chat/google_generative_ai/
# https://aistudio.google.com/prompts/new_chat?_gl=1*1g9u9qo*_ga*MTE4NjAyOTczOS4xNzM4ODIwNjQ1*_ga_P1DBVKWT6V*MTczODgzMDg1MC4yLjAuMTczODgzMDg1MC42MC4wLjEzMjM2Njk1MDI.


In [140]:
# # Create the model
# generation_config = {
#   "temperature": 1,
#   "top_p": 0.95,
#   "top_k": 64,
#   "max_output_tokens": 8192,
#   "response_mime_type": "text/plain",
# }

# model = genai.GenerativeModel(
#   model_name="gemini-2.0-flash-lite-preview-02-05",
#   generation_config=generation_config,
# )

# Use Gemini API for Response Generation

In [85]:
# genai.configure(api_key=GEMINI_API_KEY)
# model = genai.GenerativeModel("gemini-pro")

In [167]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain

In [ ]:
# Set up Pinecone vector store (adjust batch_size if necessary)
pinecone_index_name = "curehealth"

# If you're still using Pinecone for vector storage, update the index and embedding
pinecone_vector_store = PineconeVectorStore.from_documents(
    documents=text_chunks,  # Replace 'text_chunks' with actual document content
    embedding=embeddings,
    batch_size=20,  # Adjust batch size to prevent timeouts
    index_name=pinecone_index_name
)

In [89]:
# llm = GoogleGenerativeAIEmbeddings(model="gemini-pro", google_api_key=GEMINI_API_KEY)


In [169]:

system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following piences of retrieved context to answer "
    "the question.If you don't know the answer ,say that you "
    "don't know.Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system",system_prompt),
        ("human","{input}"),
    ]
)


In [121]:
# llm = OpenAI(temperature=0.4, max_tokens=400)


In [170]:
question_answer_chain = create_stuff_documents_chain(llm,prompt)
rag_chain = create_retrieval_chain(retriever,question_answer_chain)
# rag_chain = create_retrieval_chain(pinecone_vector_store,question_answer_chain)

In [ ]:
# Retrieve and Answer Query
response = rag_chain.invoke({"input": " what are the chronical diseases in a very simple terms understandble by non doctors "})
print(response["answer"])

In [152]:
retriever = docresearch.as_retriever(search_type='similarity', search_kwargs={'k': 5})


In [ ]:
system_prompt = (
    "You are an AI assistant. Use the retrieved documents to answer the question. "
    "If the answer is not in the documents, say 'The provided documents do not contain enough information.' "
    "Provide a concise and clear response."
    "\n\n"
    "{context}"
)
for doc in retrieved_docs:
    print(doc.page_content)  # Ensure the content has useful information


# New Prompt


In [ ]:
# Use a pipeline as a high-level helper
# https://console.cloud.google.com/apis/library/generativelanguage.googleapis.com?project=curehealth
from transformers import pipeline

pipe = pipeline("text-generation", model="EleutherAI/gpt-neo-2.7B")


In [ ]:
response = pipe("What is fever?")
print(response)

In [ ]:
from datasets import load_dataset

ds = load_dataset("ruslanmv/ai-medical-dataset")

In [180]:
def setup_retrieval_chain(retriever):
    system_prompt = (
        "You are an assistant for question-answering tasks. "
        "Use the following pieces of retrieved context to answer "
        "the question. If you don't know the answer, say that you "
        "don't know. Use three sentences maximum and keep the "
        "answer concise."
        "\n\n{context}"
    )
    
    prompt = ChatPromptTemplate.from_messages([("system", system_prompt), ("human", "{input}")])
    llm = OpenAI(temperature=0.4, max_tokens=400)
    question_answer_chain = create_stuff_documents_chain(llm, prompt)
    rag_chain = create_retrieval_chain(retriever, question_answer_chain)
    return rag_chain

In [181]:
def run_query(query, rag_chain):
    response = rag_chain.invoke({"input": query})
    return response["answer"]


In [182]:
def init_pinecone_vector_store(index_name, text_chunks, embeddings):
    # Initialize the Pinecone vector store
    vector_store = PineconeVectorStore.from_documents(
        documents=text_chunks,
        index_name=index_name,
        embedding=embeddings
    )
    return vector_store

In [183]:
# Set up Retriever using Pinecone Vector Store
def setup_retriever(vector_store):
    retriever = vector_store.as_retriever(search_type='similarity', search_kwargs={"k": 3})
    return retriever

In [ ]:
# Initialize Pinecone vector store and retriever
index_name = "curehealth"
vector_store = init_pinecone_vector_store(index_name, text_chunks, embeddings)
retriever = setup_retriever(vector_store)

# Setup Retrieval Chain
rag_chain = setup_retrieval_chain(retriever)

# Execute query
query = "What is Acne?"
response = run_query(query, rag_chain)
print(response)



In [ ]:

# Now, instead of calling an LLM to generate an answer, retrieve the most similar document chunks
retrieved_docs = retriever.invoke("What is Acne?")
print("Retrieved Documents:")
for doc in retrieved_docs:
    print("-" * 80)
    print(doc.page_content)
    print("-" * 80)

In [ ]:
import requests

api_key = OPENAI_API_KEY
url = "https://api.openai.com/v1/completions"

headers = {
    "Authorization": f"Bearer {api_key}",
    "Content-Type": "application/json",
}

data = {
    "model": "text-davinci-003",
    "prompt": "Say hello to the world",
    "max_tokens": 50
}

response = requests.post(url, headers=headers, json=data)

# Check if the request was successful
if response.status_code == 200:
    print("API call was successful!")
    print(response.json())  # Print the response content
else:
    print(f"Error: {response.status_code}")
    print(response.text)



In [ ]:
response = openai.Completion.create(
    model="gpt-3.5-turbo",  # or "gpt-4" if you're using that
    prompt="Say hello to the world",
    max_tokens=50
)
